Importing the required necessary libraries

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Converting the images to tensor 

In [2]:
transforms = transforms.ToTensor()

---
Splitting the data into train and test

In [3]:
train_data = datasets.MNIST(root = '../Data',train=True, download= True, transform = transforms)
test_data = datasets.MNIST(root = '../Data',train=False, download= True, transform = transforms)

Shuffling the data to avoid the overfitting

In [4]:
train_loader = DataLoader(train_data,batch_size = 10, shuffle=True)
test_loader = DataLoader(test_data,batch_size = 10, shuffle=True)

In [5]:
# Define layers
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [23]:
for i, (X_train,Y_train) in enumerate(train_data):
    break

In [30]:
x = X_train.view(1,1,28,28)
x.shape

torch.Size([1, 1, 28, 28])

In [31]:
x = F.relu(conv1(x))
x.shape

torch.Size([1, 6, 26, 26])

In [32]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 6, 13, 13])

In [33]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

Reducing the sizes of the images to reduce the computatinal speed

In [34]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 16, 5, 5])

In [36]:
x = x.view(-1, 5*5*16)
x.shape

torch.Size([1, 400])

In [7]:
conv1

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

---
->Defining the hidden layers with 120 and 84 neurons
->Using the relu activation function to learn the non-linearites
->Using the softmax to classify the number class or the label it belongs to

In [8]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [9]:
torch.manual_seed(42)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
import time
start_time = time.time()

epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1
        
        # Apply the model
        y_pred = model(X_train)  
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print interim results
        if b%600 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/60000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%')
        
    train_losses.append(loss)
    train_correct.append(trn_corr)
        
    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):

            # Apply the model
            y_val = model(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()
            
    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

epoch:  0  batch:  600 [  6000/60000]  loss: 0.04732786  accuracy:  78.450%
epoch:  0  batch: 1200 [ 12000/60000]  loss: 0.07336139  accuracy:  85.775%
epoch:  0  batch: 1800 [ 18000/60000]  loss: 0.34214985  accuracy:  88.689%
epoch:  0  batch: 2400 [ 24000/60000]  loss: 0.02389616  accuracy:  90.508%
epoch:  0  batch: 3000 [ 30000/60000]  loss: 0.00787894  accuracy:  91.663%
epoch:  0  batch: 3600 [ 36000/60000]  loss: 0.00159921  accuracy:  92.472%
epoch:  0  batch: 4200 [ 42000/60000]  loss: 0.52133870  accuracy:  93.117%
epoch:  0  batch: 4800 [ 48000/60000]  loss: 0.09050063  accuracy:  93.602%
epoch:  0  batch: 5400 [ 54000/60000]  loss: 0.00754129  accuracy:  94.024%
epoch:  0  batch: 6000 [ 60000/60000]  loss: 0.03960770  accuracy:  94.353%
epoch:  1  batch:  600 [  6000/60000]  loss: 0.00256957  accuracy:  97.900%
epoch:  1  batch: 1200 [ 12000/60000]  loss: 0.00809967  accuracy:  97.950%
epoch:  1  batch: 1800 [ 18000/60000]  loss: 0.27394569  accuracy:  97.839%
epoch:  1  b

So, it took 163 seconds for our model to train .
---

In [38]:
# Extracting the data all at once
test_load_all = DataLoader(test_data, batch_size=10000, shuffle=False)

In [39]:
with torch.no_grad():
    correct = 0
    for X_test, y_test in test_load_all:
        y_val = model(X_test)  # we don't flatten the data this time
        predicted = torch.max(y_val,1)[1]
        correct += (predicted == y_test).sum()
print(f'Test accuracy: {correct.item()}/{len(test_data)} = {correct.item()*100/(len(test_data)):7.3f}%')

Test accuracy: 9856/10000 =  98.560%


In [74]:
# Make sure to save the model only after the training has happened!
torch.save(model.state_dict(), 'Number_Predictor.pt')

In [75]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms
import numpy as np
import pandas as pd

class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [76]:
model2 = ConvolutionalNetwork()

In [77]:
model2.load_state_dict(torch.load('Number_Predictor.pt'));
model2.eval()

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)